In [ ]:
from hashlib import md5
from tqdm import tqdm
import pickle
import string
import random
import requests
import json
import time
import sys
import os
sys.path.append('../src')

In [ ]:
# Get the class names of ImageNet-1K
from open_clip import IMAGENET_CLASSNAMES

# 百度翻译API的URL
api_url = "https://fanyi-api.baidu.com/api/trans/vip/translate"

# 百度翻译API的appid和密钥
appid = ''
secretKey = ''

# 每秒请求的最大数量（QPS）
max_qps = 10


# 保存元组到文件
def save_tuple(data, filename):
    with open(filename, 'wb') as file:
        pickle.dump(data, file)


def generate_salt(length=8):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))


# 在请求中使用随机生成的salt

# 生成翻译后的中文元组


def translate_to_chinese(text_tuple, appid, secretKey, from_lang='en', to_lang='zh'):
    translated_tuple = ()
    for text in tqdm(text_tuple, desc="Translating", unit="word"):
        # 生成签名
        salt = generate_salt()
        sign = md5((appid + text + salt + secretKey).encode('utf-8')).hexdigest()

        # 构建请求参数
        params = {
            'q': text,
            'from': from_lang,
            'to': to_lang,
            'appid': appid,
            'salt': salt,
            'sign': sign
        }

        # 发送请求并获取翻译结果
        response = requests.get(api_url, params=params)
        result = response.json()

        # 提取翻译后的文本
        translated_text = result['trans_result'][0]['dst']
        translated_tuple += (translated_text,)

        # 等待一段时间以遵守QPS限制
        time.sleep(1 / max_qps)

    return translated_tuple


# 调用函数进行翻译
IMAGENET_CLASSNAMES_ZH = translate_to_chinese(
    IMAGENET_CLASSNAMES, appid, secretKey
)
print(IMAGENET_CLASSNAMES_ZH)
save_tuple(
    IMAGENET_CLASSNAMES_ZH,
    os.path.join('./results/', 'imagenet_classnames_zh.pkl')
)